# have attempted converting this kernel to PyTorch
- https://www.kaggle.com/adityaecdrid/bert-base-tf2-0-minimalistic

- **But the issue is i am not able to get that score, rather my kernel score is .219. I have tried my best to figure out that's going wrong but i am not able to do-so. 
- So Asking for help from the community as to share what did i do wrong?**

Thanks a lot for your time,
Aditya.

In [ ]:
!pip install ../input/sacremoses/sacremoses-master/ > /dev/null
!pip install ../input/transformers/transformers-master/ > /dev/null

In [ ]:
import os
import sys
import glob
import torch

import transformers
import numpy as np
import pandas as pd
import math

import torch
from transformers import BertModel
import torch, random, os, multiprocessing, glob, numpy as np, pandas as pd
import torch.nn.functional as F
from torch import nn
from torch.utils import data
from transformers import (
    BertTokenizer, BertModel, BertForSequenceClassification, 
    WEIGHTS_NAME, CONFIG_NAME
)
from tqdm import tqdm_notebook as tqdm
from transformers import AdamW
from transformers.optimization import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup

transformers.__version__

In [ ]:
train = pd.read_csv("../input/google-quest-challenge/train.csv",)
test = pd.read_csv("../input/google-quest-challenge/test.csv",)

target_cols = ['question_asker_intent_understanding', 'question_body_critical', 
               'question_conversational', 'question_expect_short_answer', 
               'question_fact_seeking', 'question_has_commonly_accepted_answer', 
               'question_interestingness_others', 'question_interestingness_self', 
               'question_multi_intent', 'question_not_really_a_question', 
               'question_opinion_seeking', 'question_type_choice',
               'question_type_compare', 'question_type_consequence',
               'question_type_definition', 'question_type_entity', 
               'question_type_instructions', 'question_type_procedure', 
               'question_type_reason_explanation', 'question_type_spelling', 
               'question_well_written', 'answer_helpful',
               'answer_level_of_information', 'answer_plausible', 
               'answer_relevance', 'answer_satisfaction', 
               'answer_type_instructions', 'answer_type_procedure', 
               'answer_type_reason_explanation', 'answer_well_written']

In [ ]:
# From the Ref Kernel's
from math import floor, ceil

def _get_masks(tokens, max_seq_length):
    """Mask for padding"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    return [1]*len(tokens) + [0] * (max_seq_length - len(tokens))

def _get_segments(tokens, max_seq_length):
    """Segments: 0 for the first sequence, 1 for the second"""
    
    if len(tokens) > max_seq_length:
        raise IndexError("Token length more than max seq length!")
        
    segments = []
    first_sep = True
    current_segment_id = 0
    
    for token in tokens:
        segments.append(current_segment_id)
        if token == "[SEP]":
            if first_sep:
                first_sep = False 
            else:
                current_segment_id = 1
    return segments + [0] * (max_seq_length - len(tokens))

def _get_ids(tokens, tokenizer, max_seq_length):
    """Token ids from Tokenizer vocab"""
    
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_ids = token_ids + [0] * (max_seq_length-len(token_ids))
    return input_ids

def _trim_input(title, question, answer, max_sequence_length=512, t_max_len=30, q_max_len=239, a_max_len=239):
    
    #293+239+30 = 508 + 4 = 512
    t = tokenizer.tokenize(title)
    q = tokenizer.tokenize(question)
    a = tokenizer.tokenize(answer)
    
    t_len = len(t)
    q_len = len(q)
    a_len = len(a)

    if (t_len+q_len+a_len+4) > max_sequence_length:
        
        if t_max_len > t_len:
            t_new_len = t_len
            a_max_len = a_max_len + floor((t_max_len - t_len)/2)
            q_max_len = q_max_len + ceil((t_max_len - t_len)/2)
        else:
            t_new_len = t_max_len
      
        if a_max_len > a_len:
            a_new_len = a_len 
            q_new_len = q_max_len + (a_max_len - a_len)
        elif q_max_len > q_len:
            a_new_len = a_max_len + (q_max_len - q_len)
            q_new_len = q_len
        else:
            a_new_len = a_max_len
            q_new_len = q_max_len
            
            
        if t_new_len+a_new_len+q_new_len+4 != max_sequence_length:
            raise ValueError("New sequence length should be %d, but is %d"%(max_sequence_length, (t_new_len + a_new_len + q_new_len + 4)))
        
        t = t[:t_new_len]
        q = q[:q_new_len]
        a = a[:a_new_len]
    
    return t, q, a

def _convert_to_bert_inputs(title, question, answer, tokenizer, max_sequence_length):
    """Converts tokenized input to ids, masks and segments for BERT"""
    
    stoken = ["[CLS]"] + title + ["[SEP]"] + question + ["[SEP]"] + answer + ["[SEP]"]

    input_ids = _get_ids(stoken, tokenizer, max_sequence_length)
    input_masks = _get_masks(stoken, max_sequence_length)
    input_segments = _get_segments(stoken, max_sequence_length)

    return [input_ids, input_masks, input_segments]

def compute_input_arays(df, columns, tokenizer, max_sequence_length):
    
    input_ids, input_masks, input_segments = [], [], []
    for _, instance in tqdm(df[columns].iterrows()):
        t, q, a = instance.question_title, instance.question_body, instance.answer
        t, q, a = _trim_input(t, q, a, max_sequence_length)
        ids, masks, segments = _convert_to_bert_inputs(t, q, a, tokenizer, max_sequence_length)
        input_ids.append(ids)
        input_masks.append(masks)
        input_segments.append(segments)
    return [
        torch.from_numpy(np.asarray(input_ids, dtype=np.int32)).long(), 
        torch.from_numpy(np.asarray(input_masks, dtype=np.int32)).long(),
        torch.from_numpy(np.asarray(input_segments, dtype=np.int32)).long(),
    ]

def compute_output_arrays(df, columns):
    return np.asarray(df[columns])

In [ ]:
tokenizer = BertTokenizer.from_pretrained("../input/pretrained-bert-models-for-pytorch/bert-base-uncased-vocab.txt")
input_categories = list(train.columns[[1,2,5]]); input_categories

In [ ]:
%%time
outputs = compute_output_arrays(train, columns = target_cols)
inputs = compute_input_arays(train, input_categories, tokenizer, max_sequence_length=512)
test_inputs = compute_input_arays(test, input_categories, tokenizer, max_sequence_length=512)

In [ ]:
inputs[0]

In [ ]:
outputs[0]

In [ ]:
test_inputs[0]

In [ ]:
%%time
lengths = np.argmax(inputs[0] == 0, axis=1)
lengths[lengths == 0] = inputs[0].shape[1]
y_train_torch = torch.tensor(train[target_cols].values, dtype=torch.float32)

In [ ]:
lengths

In [ ]:
min(lengths), max(lengths)

In [ ]:
dataset = data.TensorDataset(inputs[0], #input_ids
                             inputs[1], #input_masks
                             inputs[2], #input_segments
                             y_train_torch, #targets,
                             lengths, #lengths of each seq
                            )

In [ ]:
next(iter(dataset)) # #input_ids, #input_masks, #input_segments, #targets, #lengths of each seq

In [ ]:
BATCH_SIZE = 8
train_loader = data.DataLoader(dataset,
                               batch_size=BATCH_SIZE,
                               shuffle=True,
                               drop_last=True,
                              )

In [ ]:
next(iter(train_loader)) #input_ids, input_masks, input_segments, targets, lengths

In [ ]:
y_train_torch[0], len(y_train_torch[0])

In [ ]:
bert_model = 'bert-base-uncased'
do_lower_case = 'uncased' in bert_model
device = 'cpu'

output_model_file = 'bert_pytorch.bin'

In [ ]:
from transformers import BertConfig
bert_model_config = '../input/pretrained-bert-models-for-pytorch/bert-base-uncased/bert_config.json'
bert_config = BertConfig.from_json_file(bert_model_config)
bert_config.num_labels = 30

In [ ]:
model = BertForSequenceClassification.from_pretrained('../input/pretrained-bert-models-for-pytorch/bert-base-uncased/', config=bert_config);
model.zero_grad();
model.to(device);
'''
x = tf.keras.layers.GlobalAveragePooling1D()(sequence_output)
x = tf.keras.layers.Dropout(0.2)(x)
out = tf.keras.layers.Dense(30, activation="sigmoid", name="dense_output")(x)
''';


In [ ]:
model.classifier

In [ ]:
model

In [ ]:
epochs = 1 # had run for 3 epochs
ACCUM_STEPS = 1
optimizer = AdamW(model.parameters(), lr=3e-5, eps=4e-5)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=0.05, num_training_steps= epochs*len(train_loader)//ACCUM_STEPS)
model = model.train();

In [ ]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name)

In [ ]:
from torch.nn import CrossEntropyLoss, MSELoss, BCELoss
tr_loss = 0
import gc
criterion = nn.BCEWithLogitsLoss()

In [ ]:
%%time
temp = False
if temp:
    for _ in tqdm(range(epochs)):
        model.train();
        tk0 = tqdm(enumerate(train_loader))
        for j, batch in tk0:

            input_ids, input_masks, input_segments, y, _ = batch
            input_ids, input_masks, input_segments, y = input_ids.to(device), input_masks.to(device), input_segments.to(device), y.to(device)

            outputs = model(input_ids = input_ids.long(), 
                            labels = None, 
                            attention_mask = input_masks, 
                            token_type_ids = input_segments
                           )
            logits = outputs[0] #output preds
            loss = criterion(logits, y) #sigmoid is taken
            del input_ids, input_masks, input_segments, y
            gc.collect()

            if ACCUM_STEPS > 1:
                loss = loss / ACCUM_STEPS

            loss.backward()
            tr_loss += loss.item()

            optimizer.step()
            scheduler.step() # Update learning rate schedule # See latest docs for the order
            optimizer.zero_grad()
            global_step += 1
            gc.collect()
        torch.cuda.empty_cache()
else:
    print("Not Running As of Now..")
torch.cuda.empty_cache()
gc.collect()

In [ ]:
output_dir= "."
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

torch.save(optimizer.state_dict(), os.path.join(output_dir, 'optimizer.pt'))
torch.save(scheduler.state_dict(), os.path.join(output_dir, 'scheduler.pt'))

In [ ]:
del dataset, train_loader, lengths, optimizer, scheduler
import gc
gc.collect(), torch.cuda.empty_cache()

In [ ]:
test_inputs

In [ ]:
%%time
sequences = np.array(test_inputs[0])
lengths = np.argmax(sequences == 0, axis=1)
lengths[lengths == 0] = sequences.shape[1]

In [ ]:
lengths

In [ ]:
dataset = data.TensorDataset(test_inputs[0],
                             test_inputs[1],
                             test_inputs[2],
                             torch.from_numpy(lengths)
                            )

test_loader = data.DataLoader(dataset, 
                               batch_size=BATCH_SIZE,  
                               shuffle=False, 
                               drop_last=False
                             )

In [ ]:
next(iter(test_loader))

In [ ]:
model.eval();

In [ ]:
test_preds = np.zeros((len(test), 30)); test_preds.shape

In [ ]:
if temp:
    tk0 = tqdm(test_loader)
    for i, x_batch in enumerate(tk0):
        with torch.no_grad():
            outputs = model(input_ids = x_batch[0].to(device), 
                            labels = None, 
                            attention_mask = x_batch[1].to(device),
                            token_type_ids = x_batch[2].to(device),
                           )
            logits = outputs[0]
            test_preds[i*BATCH_SIZE : (i+1)*BATCH_SIZE] = logits.detach().cpu().squeeze().numpy()
test_pred = torch.sigmoid(torch.tensor(test_preds)).numpy()

In [ ]:
test_pred

In [ ]:
submission = pd.DataFrame.from_dict({
    'qa_id': test['qa_id']
})

for i in range(30):
    submission[target_cols[i]] = test_pred[:, i]
    
submission.to_csv(os.path.join(output_dir, 'submission.csv'), index=False)
submission.head()

In [ ]:
ls